# Escribiendo shapes

Shape Expressions (ShEx) es un lenguaje para validar y describir grafos RDF. Aunque está en proceso de estandarización, no es el estandar del W3C para esta tarea. El W3C estandar es SHACL. ShEx y SHACL no son equivalentes pero sí parecidos en intención y elementos esenciales. En este curso usaremos principalmente ShEx porque, debido a su sintaxis, seguramente nos permita aprender más rápido.

Como parte de tu tarea se solicita que realices una validación automática de RDF usando ShEx en Python. Sin embargo, probablemente no desarrollarás tu trabajo a través de Python. Estas herramientas web te resultarán de ayuda por su sencillez de uso y su capacidad para syntax highlighting:

* Editor de ShEx: https://www.weso.es/YASHE/
* Validador de ShEx: https://rdfshape.weso.es/shexValidate

El proceso de validar un grafo con ShEx requiere de 3 entradas:
* El propio grafo a validar.
* El esquema ShEx que describe la estructura que debería tener el grafo.
* Un shape map, es decir, un elemento que indica qué nodos del grafo deben conformar con qué shapes definidas en ShEx.

Vamos a trabajar con unos datos de ejemplo disponibles en el repositorio de material de la asignatura:


In [ ]:
!git clone https://github.com/cursosLabra/miw_websem2425.git

Cloning into 'miw_websem2425'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 37 (delta 13), reused 27 (delta 8), pack-reused 0 (from 0)
Receiving objects: 100% (37/37), 48.48 KiB | 689.00 KiB/s, done.
Resolving deltas: 100% (13/13), done.


In [ ]:
!pip install rdflib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.9/564.9 kB 10.7 MB/s eta 0:00:00


In [ ]:
from rdflib import Graph
g = Graph()
base_path = "/content/miw_websem2425/lab_sessions/"
g.parse(base_path + "books_data.ttl")
print(g.serialize(format="turtle"))

@prefix dcterms: <http://purl.org/dc/terms/> .
@prefix ex: <http://example.org/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix schema1: <http://schema.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ex:book1 a schema1:Book ;
    dcterms:title "The Enchanted Forest" ;
    schema1:ISBN "978-3-16-148410-0" ;
    schema1:author ex:author1 ;
    schema1:datePublished "2005-06-20"^^xsd:date ;
    schema1:genre ex:genre_fantasy ;
    schema1:publisher ex:publisher1 ;
    schema1:translatedInto "Spanish" .

ex:book2 a schema1:Book ;
    dcterms:title "Stars Beyond Reach" ;
    schema1:ISBN "978-1-23-456789-0" ;
    schema1:author ex:author2 ;
    schema1:datePublished "2010-09-15"^^xsd:date ;
    schema1:genre ex:genre_scifi ;
    schema1:publisher ex:publisher2 ;
    schema1:translatedInto "French" .

ex:book3 a schema1:Book ;
    dcterms:title "The Hidden Cipher" ;
    schema1:ISBN "978-0-98-765432-1" ;
    schema1:author ex:author3 ;
    schema1:datePublished "2018-03-1

Para validar nuestro grafo, nosotros, como expertos de dominio, tenemos que saber qué estructuras topológicas deben darse en nuestro grafo (y describirlas con ShEx). En palabras llanas: qué propiedades se supone que vamos a asociar a cada tipo de nodo y qué esperamos encontrar del otro lado de esas propiedades.

No es obligatorio, pero sí habitual y natural asociar una clase a una shape. Si describimos una clase para una shape lo que estaremos diciendo es que aquellos nodos que sean instancia de esa clase deben tener un vecindario inmediato que se ajuste en propiedades y nodos/literales conectados a lo que se indica para la shape de esa clase. Por ejemplo, si escribimos en una shape que la clase libro debería tener un autor y un género, lo que queremos decir es que todas las instancias de libro deberían tener su propio autor y su propio género.

En ShEx, eso se expresaría de la siguiente manera (para verlos con syntax highlighting, te recomiendo que copiar estos ejemplos en YASHE):



In [ ]:
str_shex_basica_para_libro = """
PREFIX :       <http://example.org/>
PREFIX schema: <http://schema.org/>

:shape_libro {
  a [schema:Book] ;  # Indico que estos nodos tiene que tener como valor de rdf:type exactamente la URI schema:Book.
  schema:author IRI ; # Indico que estos nodos tienen que tener exactamente un schema:author. Y el objeto de esta tripleta debe ser una IRI (no un BNode o un Literal)
  schema:genre IRI
}

"""

Esta shape, en realidad, está diciendo que los libros tienen que tener una porpiedad schema:author y otra schema:genre, pero no dice sí los nodos al otro lado de esa propiedad tienen que ser de tipo Persona o Género necesariamente. Para eso, lo más adecuado, sería definir también uans shapes para esos conceptos e indicar que, en lugar de IRI, los nodos al otro lado de esas propiedades deben de ser nodos que conformen con las shapes de Persona y Género:

In [ ]:
str_2_shapes = """
PREFIX :       <http://example.org/>
PREFIX schema: <http://schema.org/>

:shape_libro {
  a [schema:Book] ;
  schema:author @:shape_Author ;
  schema:genre @:shape_Genre
}

:shape_Author {
   a [schema:Person] ;
}

:shape_Genre {
   a [schema:Genre] ;
}

"""

Este esquema aún no expresa todo lo que necesitamos expresar ni sobre libro ni sobre muchas otras entidades de nuestro grafo. Pero avanza en la dirección adecuada. Ya no vale cualqueir URI para poner como autor de un libro: la URI dle nodo que sea debe conformar con la shape_Author, es decir, tiene que ser de tipo persona.

Otras características de la shape Libro, con los datos que manejamos, serían las siguientes:


In [ ]:
shex_completa_libro = """
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX :       <http://example.org/>
PREFIX schema: <http://schema.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>


:shape_libro {
  a [schema:Book] ;
  schema:author @:shape_Author ;
  schema:genre @:shape_Genre + ; # Cambio de cardinalidad: un libro tiene entre 1 y muchos géneros.
  dcterms:title xsd:string ; # Un título que apunte a un literal de tipo string.
  schema:datePublished xsd:date ; # fecha de publicación que apunte a un literal fecha.
  schema:ISBN xsd:string ? # Un libro tiene entre 0 y 1 códigos ISBN (de tipo string)
}

:shape_Author {
   a [schema:Person] ;
}

:shape_Genre {
   a [schema:Genre] ;
}
"""

Con esto ya tenemos un grafo y un esquema (aún in completo, pero válido) para validar. Nos queda el shape map. El siguiente shape map indica que queremos comprobar si el nodo ex:book1 conforma con la shape de libro. Si copiar el contenido turtle, el esquema ShEx y el shape map en sus respectivos campos en RDFshape, podrás ejecutar una validación de grafo.

In [ ]:
shape_map_book1_conforma_libro = """

<http://example.org/book1>@<http://example.org/shape_libro>

"""

Si has ejecutado esto en RDFShape, verás que los resultados de validación son para el nodo book1 pero también para author1 y genre_fantasy. Esto ha sido necesario porque, para confirmar que book1 es un shape_libro correcto, también se hubo de determinar si el género y autor con los que está conectado conforman con las shapes que deben conformar.

Podríamos añadir más nodos a la validación separando secuencias como la del anterior shape map separadas por comas. Por ejemplo:


In [ ]:
shape_map_varios_libros_conforman_libro = """

<http://example.org/book1>@<http://example.org/shape_libro>,
<http://example.org/book2>@<http://example.org/shape_libro>,
<http://example.org/book3>@<http://example.org/shape_libro>

"""

Pero, obviamente, esto se vuelve impracticable en un caso real en el que queramos validar muchso nodos. Para eso tenemos unas asociaciones nodos-shape alternativas: las expresiones FOCUS. Estas expresiones definen un patrón de grafo con una pequeña consulta. Por ejemplo:

In [ ]:
shape_map_todos_libros="""

{FOCUS a <http://schema.org/Book>}@<http://example.org/shape_libro>

"""

El anterior shape_map sirve para validar TODOS los libros, entendiéndose por libro todo aquel nodo que en sea el sujeto de tripletas donde el predicado sea rdf:type y el objeto schema:Book. Es decir, todas las instancias de libro. En este patrón, los nodos objetivo de la validación son aquellos que encajen con la palabra clave FOCUS en el patrón de grafo descrito.

Se pueden crear shape maps que combinen asociaciones de expresiones FOCUS a shapes con asignaciones de nodos individuales a shapes:

In [ ]:
shape_map_todos_libros_y_un_autor = """
{FOCUS a <http://schema.org/Book>}@<http://example.org/shape_libro>,
<http://example.org/author1>@<http://example.org/shape_Author>
"""

La manera de hacer esta evaluación en Python es a través de una librería llamada PyShex. Desafortunadamente, pyshex no ofrece soporte completo a los shape maps. En cada validación, debemos pasar como parámetro el grafo, el esquema ShEx, la URI de un nodo que queremos validar y la URI de la shape contra la que lo queremos validar. A continuación tienes un ejemplo de uso de la librería. No obstante, en su repositorio de github, podrás encontrar otros Juoyter con más ejemplos: https://github.com/hsolbrig/PyShEx/tree/master/notebooks

In [ ]:
!pip install pyshex

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.8/69.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 7.1 MB/s eta 0:00:00
  Created wheel for cfgraph: filename=CFGraph-0.2.1-py3-none-any.whl size=2214 sha256=f574cbb2906d5495376de71b4061224491e51f36258a63cab27fd5a893353288
  Stored in directory: /root/.cache/pip/wheels/8b/ac/8f/0f5b041561f10e6db39796e4ad11011b8f5a4b0a0ed74a5e61
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144555 sha256=f6803f2c973c4067b3a798975afdf7a4af1cdd69214fe249c997d71fc765ff4c
  Stored in directory: /root/.cache/pip/wheels/1a/97/32/461f837398029ad76911109f07047fde1d7b661a147c7c56d1
Successfully built cfgraph antlr4-python3-runtime


In [ ]:
from pyshex import ShExEvaluator

results = ShExEvaluator().evaluate(g.serialize(format="turtle"), # grafo
                                   shex_completa_libro,          # esquema shex
                                   focus="http://example.org/book1", # URI del nodo que vamos a evaluar
                                   start="http://example.org/shape_libro") # shape de inicio contra la que enfrentar el foco
for r in results:
    if r.result:
        print("PASS")
    else:
        print(f"FAIL:\n {r.reason}")

PASS


Para realizar una validación completa de un dataset con pyshex, deberás llamar al evaluate() varias veces. Para rescatar todos los nodos de un tipo, seguramente lo más sencillo será que uses el método de rdflib que te permite rápidamente iterar sobre las instancias de cierta clase (consulta notebooks pasados sobre el manejo de RDF en Python con rdflib).